In [1]:
import pandas as pd
import numpy as np
import nltk
import nltk.sentiment.util
from nltk.tokenize import sent_tokenize, word_tokenize
import matplotlib.pyplot as plt
import pprint
import re
import gensim
import gensim.corpora as corpora
import os
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import string
import math

In [2]:
df = pd.read_csv("amazon_reviews_us_Grocery_v1_00_sentiment.csv").dropna().sample(50000)
df.set_index("review_id")

,marketplace,customer_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,polarity,subjectivity
review_id,,,,,,,,,,,,,,,,
R1X5DSHEYGV3HQ,US,41283779,B0000D9MVD,383599191,Carli Balsamic Vinegar of Modena (250 ml),Grocery,5,0,0,N,Y,Excellent,Excellent.,2015-04-29,1.000000,1.000000
R1RLZ2277AQJXA,US,11859992,B0076MBIL0,286167785,Dried Cherries,Grocery,5,0,0,N,Y,Only dried cherries,Nice flavor and no sugar!! These are great wh...,2014-03-12,0.624306,0.808333
R2HFWEX37IL93S,US,27036965,B001QK44PK,77621591,"Bali's Best Citrus Green Tea Candy, 5.3-Ounce ...",Grocery,5,0,0,N,Y,worthy the price and the hazard to order it on...,Everybody in my office love the candy. Not ver...,2015-04-01,0.282778,0.602906
R3CHV1E2K9GYVL,US,46384901,B004EDUVHQ,360435814,"Green Mountain Coffee Breakfast Blend Decaf, ...",Grocery,5,1,1,N,Y,Great {rice,"This was a great buy, I usually don't buy deca...",2013-02-11,0.137500,0.250000
RZZ0IZHSAWJGQ,US,17655204,B001EQ4ZGO,266829191,"Kool-Aid Sugar Free On-the-Go Cherry, 0.5-Ounc...",Grocery,5,0,0,N,Y,Kool-Aid,I absolutely love this drink mix which is why ...,2012-12-14,0.238889,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RPKOM1JGCQ9QV,US,11463513,B0077T5S5O,424222099,Starbucks Hot Cocoa Mix (pack of 8),Grocery,5,0,0,N,Y,Five Stars,"Good, very very",2014-08-02,0.480000,0.495000
R1XF6YJFSWGWOG,US,32726452,B001VNEDDA,369674070,"Frontier Garlic Powder, 16 Ounce Bags (Pack of 2)",Grocery,4,5,7,N,Y,Garlicky!,"Lots of it, even for someone like me who uses ...",2012-12-12,0.106250,0.237500
R3BDSNPJBARU68,US,11666350,B0009F3SFK,714359203,"Yogi Teas Woman's Mother To Be, 16 Count (Pack...",Grocery,4,1,4,N,Y,Trying to Conceive?,I see a lot of reviews about how this tea is g...,2012-02-02,0.116964,0.282738


In [3]:
data = df.drop(["marketplace", "customer_id", "product_parent", "product_category", "review_date"], axis=1)
data["health_hazard"] = np.nan

In [4]:
neg_reviews = data.query("star_rating < 4", engine="python").set_index("review_id")

In [5]:
stop_words = stopwords.words('english')
stop_words.extend(['br'])
translator=str.maketrans('','',string.punctuation)

neg_reviews["tokened_review"] = np.nan
for index, row in neg_reviews.iterrows():
    text = row["review_body"].lower()
    text = text.translate(translator)
    text = word_tokenize(text)
    new_text = []
    for token in text:
        if token not in stop_words:
            new_text.append(token)
    neg_reviews.loc[index, "tokened_review"] = str(new_text)
    

In [6]:
hazardous_words=["rotten", "mouldy", "moldy", "mold", "mould", "sick", "dangerous", "diarrhea", "poisoning", "stale"]
for word in hazardous_words:
    neg_reviews.loc[[word in tokened_review for tokened_review in neg_reviews["tokened_review"]], "health_hazard"] = 1
    

In [7]:
num_labeled = len(neg_reviews.query("health_hazard == 1", engine="python"))
num_labeled

419

In [8]:
neg_reviews.query("health_hazard == 1", engine="python")

,product_id,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,polarity,subjectivity,health_hazard,tokened_review
review_id,,,,,,,,,,,,,
R34AMA24C4T830,B005ZBZM52,"San Francisco Bay Coffee, OneCup Single Serve ...",3,25,34,N,Y,Weak at High Setting,This product is an interesting cross between a...,0.008869,0.227282,1.0,"['product', 'interesting', 'cross', 'keurig', ..."
R2NKOS9IWIB7E1,B006J1FBLM,"Haribo SUGAR FREE Classic Gummi Bears, 1 Lb",1,35,47,N,N,Weaponized Snack Foods & Criminal Intent,Ooh! Colorful Sugar FREE gummies... I didn...,0.035662,0.346136,1.0,"['ooh', 'colorful', 'sugar', 'free', 'gummies'..."
R3PPJBZBUQFWDZ,B000G671RI,Fried Pork Rinds Salt & Vinegar 12 bags (1.75oz),1,0,0,N,Y,One Star,"Super stale, they all ended up in the trash.",-0.083333,0.583333,1.0,"['super', 'stale', 'ended', 'trash']"
R1WCZ8F8F605W9,B004J0NPD6,"Jelly Belly Jelly Beans, Assorted, 1 Pound",1,21,21,N,Y,Are these realy Jelly Belly jelly beans??,Nothing on the packaging identifies this produ...,0.207619,0.494127,1.0,"['nothing', 'packaging', 'identifies', 'produc..."
R111Q4GCE14E1W,B005JDEKD6,"Claeys Sanded Candy Drops, 2 Pound",1,1,1,N,Y,Sugary sugar sugar crystal with a hint of arti...,"Not even close to the description: \\""the swee...",-0.254167,0.554167,1.0,"['even', 'close', 'description', 'sweet', 'tas..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
R22NXGOQPM12C8,B000EDG598,Super-Fine Almond Flour,1,21,30,N,Y,Not 100% safe for SCD/GAPS!,BRM 'gluten free' items still contain some sor...,-0.078254,0.390822,1.0,"['brm', 'gluten', 'free', 'items', 'still', 'c..."
R3F7VL99XI63D6,B000EQTAVA,Terra Chips,3,0,0,N,Y,Some of the packages were stale.,Some of the packages in the case were stale. T...,0.125000,0.587500,1.0,"['packages', 'case', 'stale', 'good', 'ones', ..."
R4NQH5BBPZNXW,B004P7RGMY,"Nespresso OriginalLine, Arpeggio, 10 Count",1,1,1,N,Y,odd taste,"Stale and came smashed up. Poor quality, from ...",-0.350000,0.416667,1.0,"['stale', 'came', 'smashed', 'poor', 'quality'..."


In [9]:
#neg_reviews.loc["R2DQQR4ABND97M", "tokened_review"]

In [10]:
num_labeled = len(neg_reviews.query("health_hazard == 1", engine="python"))
num_labeled

419

In [11]:
neg_reviews.query("health_hazard == 0", engine="python")

,product_id,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,polarity,subjectivity,health_hazard,tokened_review
review_id,,,,,,,,,,,,,


In [12]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

In [13]:
neg_reviews.query("health_hazard == 0 or health_hazard == 1", engine="python")

,product_id,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,polarity,subjectivity,health_hazard,tokened_review
review_id,,,,,,,,,,,,,
R34AMA24C4T830,B005ZBZM52,"San Francisco Bay Coffee, OneCup Single Serve ...",3,25,34,N,Y,Weak at High Setting,This product is an interesting cross between a...,0.008869,0.227282,1.0,"['product', 'interesting', 'cross', 'keurig', ..."
R2NKOS9IWIB7E1,B006J1FBLM,"Haribo SUGAR FREE Classic Gummi Bears, 1 Lb",1,35,47,N,N,Weaponized Snack Foods & Criminal Intent,Ooh! Colorful Sugar FREE gummies... I didn...,0.035662,0.346136,1.0,"['ooh', 'colorful', 'sugar', 'free', 'gummies'..."
R3PPJBZBUQFWDZ,B000G671RI,Fried Pork Rinds Salt & Vinegar 12 bags (1.75oz),1,0,0,N,Y,One Star,"Super stale, they all ended up in the trash.",-0.083333,0.583333,1.0,"['super', 'stale', 'ended', 'trash']"
R1WCZ8F8F605W9,B004J0NPD6,"Jelly Belly Jelly Beans, Assorted, 1 Pound",1,21,21,N,Y,Are these realy Jelly Belly jelly beans??,Nothing on the packaging identifies this produ...,0.207619,0.494127,1.0,"['nothing', 'packaging', 'identifies', 'produc..."
R111Q4GCE14E1W,B005JDEKD6,"Claeys Sanded Candy Drops, 2 Pound",1,1,1,N,Y,Sugary sugar sugar crystal with a hint of arti...,"Not even close to the description: \\""the swee...",-0.254167,0.554167,1.0,"['even', 'close', 'description', 'sweet', 'tas..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
R22NXGOQPM12C8,B000EDG598,Super-Fine Almond Flour,1,21,30,N,Y,Not 100% safe for SCD/GAPS!,BRM 'gluten free' items still contain some sor...,-0.078254,0.390822,1.0,"['brm', 'gluten', 'free', 'items', 'still', 'c..."
R3F7VL99XI63D6,B000EQTAVA,Terra Chips,3,0,0,N,Y,Some of the packages were stale.,Some of the packages in the case were stale. T...,0.125000,0.587500,1.0,"['packages', 'case', 'stale', 'good', 'ones', ..."
R4NQH5BBPZNXW,B004P7RGMY,"Nespresso OriginalLine, Arpeggio, 10 Count",1,1,1,N,Y,odd taste,"Stale and came smashed up. Poor quality, from ...",-0.350000,0.416667,1.0,"['stale', 'came', 'smashed', 'poor', 'quality'..."


In [14]:
labeled = neg_reviews.query("health_hazard == 0 or health_hazard == 1", engine="python")
reviews = labeled.loc[:, 'tokened_review'].values
y = labeled.loc[:, 'health_hazard'].values
for i in range(len(reviews)):
    reviews[i] = stemmer.stem(reviews[i])

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
matrix = CountVectorizer(max_features=100)
X = matrix.fit_transform(reviews).toarray()
X

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 3, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [16]:
X.shape

(419, 100)

In [17]:
# split train and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [18]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()

#from sklearn.linear_model import LogisticRegression

#classifier = LogisticRegression()

classifier.fit(X_train, y_train)

# Predict Class
y_pred = classifier.predict(X_test)

# Accuracy 
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

1.0


In [19]:
trial = neg_reviews.sample(100)

In [20]:
matrix.get_feature_names()

['almost',
 'also',
 'amazon',
 'another',
 'arrived',
 'away',
 'back',
 'bad',
 'bag',
 'bags',
 'batch',
 'better',
 'bought',
 'box',
 'brand',
 'buy',
 'candy',
 'cant',
 'chocolate',
 'coffee',
 'company',
 'could',
 'date',
 'didnt',
 'disappointed',
 'dont',
 'eat',
 'eating',
 'even',
 'ever',
 'expiration',
 'find',
 'first',
 'flavor',
 'food',
 'free',
 'fresh',
 'get',
 'give',
 'good',
 'got',
 'great',
 'hard',
 'however',
 'id',
 'im',
 'ingredients',
 'ive',
 'know',
 'like',
 'little',
 'long',
 'love',
 'made',
 'many',
 'maybe',
 'mold',
 'moldy',
 'much',
 'natural',
 'never',
 'old',
 'one',
 'opened',
 'order',
 'ordered',
 'package',
 'price',
 'product',
 'purchased',
 'quality',
 'really',
 'received',
 'recommend',
 'see',
 'sick',
 'since',
 'smell',
 'something',
 'stale',
 'still',
 'store',
 'sugar',
 'sure',
 'sweet',
 'taste',
 'tasted',
 'think',
 'though',
 'thought',
 'time',
 'tried',
 'try',
 'two',
 'use',
 'way',
 'well',
 'wont',
 'would',
 'yea

In [21]:
trial = neg_reviews.sample(100)
trial_x = trial.loc[:,"tokened_review"].values
trial_X = matrix.fit_transform(trial_x).toarray()
trial_y = classifier.predict(trial_X)
trial_y

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [22]:
trial["review_body"][8]

'I typically don\'t mind fishy snacks, but the fishiness in this snack tastes very much unbalanced against the sweetness. Perhaps a milder scent would have been better, or maybe if they had taken out the sweetness altogether.<br /><br />Also, the cashews are not clustered as they are pictured, but individual \\\\"free agents\\\\" in the bag.'